In [16]:
import os
import sys
import cv2
import pandas as pd
from glob import glob
from PIL import Image
from enum import Enum
from typing import Optional, Tuple, List
from sklearn.model_selection import GroupShuffleSplit, train_test_split

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

ROOT_DIR = 'D:/Projects/Skin-Disease-Detection-Pytorch'
sys.path.append(ROOT_DIR)
from src.utils.config import *
from src.utils.dataloader import *
from src.utils.config import *

In [97]:
class DermnetDataset(Dataset):
    def __init__(
            self, 
            data: Tuple[str, int], 
            transform: Optional[transforms.Compose] = None
    ) -> None:
        self.data = data
        self.transform = transform

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor]:
        image_path, label = self.data[index]
        image = Image.open(image_path)
        label = torch.tensor(label)
        label = F.one_hot(label, num_classes=len(DERMNET_LABEL_NAME)).float()
        

        if self.transform:
            image = self.transform(image)
        return image, label, image_path



def _transforms(resize_size, crop_size):
    train_transform = transforms.Compose(
            [
                # transforms.ToPILImage(),
                transforms.Resize(resize_size[:-1]),
                transforms.CenterCrop(crop_size[:-1]),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ]
        )
    val_transform = transforms.Compose(
            [
                # transforms.ToPILImage(),
                transforms.Resize(resize_size[:-1]),
                transforms.CenterCrop(crop_size[:-1]),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ]
        )
    
    test_transform= transforms.Compose(
            [
                # transforms.ToPILImage(),
                transforms.Resize(resize_size[:-1]),
                transforms.CenterCrop(crop_size[:-1]),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ]
        )
    
    return train_transform, val_transform, test_transform
    



def prepare_dermnet_data(
        resize_size: Tuple[int, int, int], 
        crop_size: Tuple[int, int, int], 
        batch_size: int, 
        num_workers: int
) -> Tuple[DataLoader, DataLoader, DataLoader]:
    
    DERMNET_DATA_DIR = 'D:/Datasets/Skin-Disease-Detection/dataset/Dermnet'
    label_names = os.listdir(f'{DERMNET_DATA_DIR}/train')
    train_data = []
    val_data = []
    for label in label_names:
        file_paths = glob(f'{DERMNET_DATA_DIR}/train/{label}/*')
        train_paths, val_paths = train_test_split(file_paths, test_size=0.2, random_state=42)

        sparse_label = label_names.index(label)
        train_data += [(path, sparse_label) for path in train_paths]
        val_data += [(path, sparse_label) for path in val_paths]

    test_data = []
    for label in label_names:
        file_paths = glob(f'{DERMNET_DATA_DIR}/test/{label}/*')
        sparse_label = label_names.index(label)
        test_data += [(path, sparse_label) for path in file_paths]


    train_transform, val_transform, test_transform = _transforms(resize_size[:-1], crop_size[:-1])

    train_ds = DermnetDataset(data=train_data, transform=train_transform)
    val_ds = DermnetDataset(data=val_data, transform=val_transform)
    test_ds = DermnetDataset(data=test_data, transform=test_transform)

    train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        persistent_workers=True,
        pin_memory=True
    )

    val_dataloader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        persistent_workers=True,
        pin_memory=True
    )

    test_dataloader = DataLoader(
        test_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        persistent_workers=True,
        pin_memory=True
    )

    return train_dataloader, val_dataloader, test_dataloader

In [100]:
DERMNET_DATA_DIR = 'D:/Datasets/Skin-Disease-Detection/dataset/Dermnet'
label_names = os.listdir(f'{DERMNET_DATA_DIR}/train')
train_data = []
val_data = []
for label in label_names:
    file_paths = glob(f'{DERMNET_DATA_DIR}/train/{label}/*')
    train_paths, val_paths = train_test_split(file_paths, test_size=0.2, random_state=42)

    sparse_label = label_names.index(label)
    train_data += [(path, sparse_label) for path in train_paths]
    val_data += [(path, sparse_label) for path in val_paths]

# test_data = []
# for label in label_names:
#     file_paths = glob(f'{DERMNET_DATA_DIR}/test/{label}/*')
#     sparse_label = label_names.index(label)
#     test_data += [(path, sparse_label) for path in file_paths]


train_transform, val_transform, test_transform = _transforms((256, 256), (224, 224))

train_ds = DermnetDataset(data=train_data, transform=train_transform)
# val_ds = DermnetDataset(data=val_data, transform=val_transform)
# test_ds = DermnetDataset(data=test_data, transform=test_transform)

img, label, image_path = train_ds.__getitem__(3200)
print(img.shape, label, image_path)

tensor(5)
torch.Size([3, 224, 224]) tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]) D:/Datasets/Skin-Disease-Detection/dataset/Dermnet/train/Eczema Photos\lichen-simplex-chronicus-114.jpg


In [91]:
import torch
import torch.nn.functional as F
import numpy as np

# Your continuous label tensor
label = torch.from_numpy(np.arange(5, 10))

# Convert to integer indices
label_indices = (label.float() - label.min().float()).long()
label_indices
# Apply F.one_hot
# one_hot_label = F.one_hot(label_indices, num_classes=7)

# print(one_hot_label)


tensor([0, 1, 2, 3, 4])

In [113]:
def get_base_model(base_model_name):
    assert base_model_name in [
        'resnet18',
        'resnet50',
        'efficientnet_b0', 
        'efficientnet_b1', 
        'efficientnet_b2', 
        'efficientnet_b3', 
        'efficientnet_b4', 
        'efficientnet_v2_s', 
        'efficientnet_v2_m'
    ], f'Invalid base model name: {base_model_name}'

    return (
        resnet18(weights=None) if base_model_name == 'resnet18' else
        resnet50(weights=None) if base_model_name == 'resnet50' else
        efficientnet_b0(weights=None) if base_model_name == 'efficientnet_b0' else
        efficientnet_b1(weights=None) if base_model_name == 'efficientnet_b1' else
        efficientnet_b2(weights=None) if base_model_name == 'efficientnet_b2' else
        efficientnet_b3(weights=None) if base_model_name == 'efficientnet_b3' else
        efficientnet_b4(weights=None) if base_model_name == 'efficientnet_b4' else
        efficientnet_v2_s(weights=None) if base_model_name == 'efficientnet_v2_s' else
        efficientnet_v2_m(weights=None)
    )

In [125]:
# class Network(nn.Module):
#     def __init__(
#             self, 
#             base_model, 
#             dropout: float, 
#             output_dims: List[int],
#             num_classes: int
#     ) -> None:
#         super().__init__()

#         self.base_model = base_model
#         input_dim: int = base_model.classifier[1].in_features

#         layers: List[nn.Module] = []
#         for output_dim in output_dims:
#             layers.append(nn.Linear(input_dim, output_dim))
#             layers.append(nn.ReLU())
#             layers.append(nn.Dropout(dropout))
#             input_dim = output_dim
#         layers.append(nn.Linear(input_dim, num_classes))
#         layers.append(nn.Softmax(dim=-1))

#         self.base_model.classifier = nn.Sequential(*layers)
    
#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         return self.base_model(x)
    

# class ResNetwork(nn.Module):
#     def __init__(
#             self, 
#             base_model, 
#             dropout: float, 
#             output_dims: List[int],
#             num_classes: int
#     ) -> None:
#         super().__init__()

#         self.base_model = base_model
#         input_dim: int = base_model.fc.in_features

#         layers: List[nn.Module] = []
#         for output_dim in output_dims:
#             layers.append(nn.Linear(input_dim, output_dim))
#             layers.append(nn.ReLU())
#             layers.append(nn.Dropout(dropout))
#             input_dim = output_dim
#         layers.append(nn.Linear(input_dim, num_classes))
#         layers.append(nn.Softmax(dim=-1))

#         self.base_model.fc = nn.Sequential(*layers)
    
#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         return self.base_model(x)
    

class Network(nn.Module):
    def __init__(
            self, 
            base_model, 
            dropout: float, 
            output_dims: List[int],
            num_classes: int
    ) -> None:
        super().__init__()

        self.base_model = base_model
        if base_model.__class__.__name__ == 'ResNet':
            input_dim: int = base_model.fc.in_features 
        elif base_model.__class__.__name__ == 'EfficientNet':
            input_dim: int = base_model.classifier[1].in_features

        layers: List[nn.Module] = []
        for output_dim in output_dims:
            layers.append(nn.Linear(input_dim, output_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = output_dim
        layers.append(nn.Linear(input_dim, num_classes))
        layers.append(nn.Softmax(dim=-1))


        if base_model.__class__.__name__ == 'ResNet':
            self.base_model.fc = nn.Sequential(*layers) 
        elif base_model.__class__.__name__ == 'EfficientNet':
            self.base_model.classifier = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.base_model(x)

In [ ]:
base_model_name = 'resnet50'
base_model_name = 'efficientnet_b0'


model = get_base_model(base_model_name)

MODEL = Network(
    base_model=get_base_model(base_model_name),
    dropout=None,
    output_dims=[],
    num_classes=len(DERMNET_LABEL_NAME)
) 
MODEL

In [9]:
train_data, val_data, test_data = prepare_dermnet_data()
len(train_data), len(val_data), len(test_data)

(12438, 3119, 4002)

In [15]:
train_dataloader, _, _ = local_dataloader(
    resize_size=(256, 256, 3),
    crop_size=(224, 224, 3),
    batch_size=10,
    num_workers=2
)

for image, label in train_dataloader:
    print(image.shape, label.shape)
    break

torch.Size([10, 3, 224, 224]) torch.Size([10, 3])
